In [1]:
! pip install pycocotools
! pip install -U ultralytics

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 1.1/1.1 MB 54.6 MB/s  0:00:00

  Attempting uninstall: ultralytics-thop

    Found existing installation: ultralytics-thop 2.0.17

    Uninstalling ultralytics-thop-2.0.17:

      Successfully uninstalled ultralytics-thop-2.0.17

  Attempting uninstall: ultralytics

    Found existing installation: ultralytics 8.3.221

    Uninstalling ultralytics-8.3.221:

      Successfully uninstalled ultralytics-8.3.221

   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   -------------------- ------------------- 1/2 [ultralytics]
   ---------------------------------------- 2/2 [ultralytics]



In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import kagglehub
import torchvision
import pandas as pd
import torchvision.tv_tensors as tv_tensors


from tqdm import tqdm
from torch.utils.data import DataLoader, WeightedRandomSampler
from PIL import Image, ImageEnhance, ImageFilter
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
from torchvision.transforms import v2
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torchvision.models.detection.ssd import SSD300_VGG16_Weights

In [2]:
# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.mps.is_available() else device)
print(f"Using device: {device}")

Using device: cuda


## 하이퍼 파라미터 조정

# 1차 데이터

## 데이터 로드

In [17]:
path = "C:/Users/main/Downloads/final_dataset"
train_dir = os.path.join(path, "images")
annotation_dir = os.path.join(path, "final_annotations.json")

In [ ]:
img_files = [f for f in os.listdir(train_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"Train 수: {len(img_files)}")

Train 수: 3228


## 함수 생성

In [18]:
import json
from collections import defaultdict

# JSON 로드
with open(annotation_dir, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# ✅ 이미지별 어노테이션 매핑 생성
annotation_mapping = defaultdict(list)
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    image_name = next(
        (img["file_name"] for img in coco_data["images"] if img["id"] == image_id),
        None
    )
    if image_name:
        annotation_mapping[image_name].append(ann)

print("✅ annotation_mapping 생성 완료")
print("예시 키:", list(annotation_mapping.keys())[:5])
print("첫 번째 이미지의 어노테이션 수:", len(annotation_mapping[list(annotation_mapping.keys())[0]]))
print(annotation_mapping)



✅ annotation_mapping 생성 완료
예시 키: ['K-001900-016548-018110-021026_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-021026_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-027926_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_90_000_200.png']
첫 번째 이미지의 어노테이션 수: 4
defaultdict(<class 'list'>, {'K-001900-016548-018110-021026_0_2_0_2_70_000_200.png': [{'area': 28341, 'iscrowd': 0, 'bbox': [173, 279, 201, 141], 'category_id': 1899, 'ignore': 0, 'segmentation': [], 'id': 16, 'image_id': 833}, {'area': 57600, 'iscrowd': 0, 'bbox': [140, 866, 240, 240], 'category_id': 16547, 'ignore': 0, 'segmentation': [], 'id': 19, 'image_id': 833}, {'area': 98838, 'iscrowd': 0, 'bbox': [628, 845, 323, 306], 'category_id': 18109, 'ignore': 0, 'segmentation': [], 'id': 22, 'image_id': 833}, {'area': 30448, 'iscrowd': 0, 'bbox': [674, 274, 176, 173], 'category_id': 21025, 'ignore': 0, 'segmentation': [], 'id': 24, 'image_id': 833}], 'K-0019

In [19]:
from sklearn.model_selection import train_test_split

# 전체 이미지 파일 이름 리스트
image_names = list(annotation_mapping.keys())

# 이미지 리스트를 train/val로 나누기
train_list, val_list = train_test_split(image_names, test_size=0.2, random_state=42)

# 어노테이션 매핑도 함께 나누기
annotation_mapping_train = {name: annotation_mapping[name] for name in train_list}
annotation_mapping_val   = {name: annotation_mapping[name] for name in val_list}

print("✅ train/val 분리 완료")
print(f"Train 이미지 수: {len(train_list)}")
print(f"Val 이미지 수: {len(val_list)}")


✅ train/val 분리 완료
Train 이미지 수: 3052
Val 이미지 수: 763


## YOLO 변환 코드

In [11]:
def save_coco_split_json(save_path, mapping, split_name):
    images = []
    annotations = []
    categories = coco_data["categories"]

    for i, name in enumerate(mapping.keys()):
        img_info = next((img for img in coco_data["images"] if img["file_name"] == name), None)
        if img_info:
            new_img = img_info.copy()
            new_img["id"] = i + 1
            images.append(new_img)

            for ann in mapping[name]:
                new_ann = ann.copy()
                new_ann["image_id"] = new_img["id"]
                annotations.append(new_ann)

    coco_split = {
        "images": images,
        "annotations": annotations,
        "categories": categories
    }

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(coco_split, f, ensure_ascii=False, indent=2)
    print(f"✅ {split_name} JSON 저장 완료 → {save_path}")

In [12]:
def coco_to_yolo(json_path, split, yolo_root):
    with open(json_path, "r", encoding="utf-8") as f:
        coco = json.load(f)

    # ✅ 카테고리 매핑 테이블 생성 (category_id → YOLO 인덱스)
    category_id_to_index = {cat["id"]: i for i, cat in enumerate(coco["categories"])}

    images = {img["id"]: img for img in coco["images"]}
    ann_by_img = {img_id: [] for img_id in images.keys()}

    for ann in coco["annotations"]:
        ann_by_img[ann["image_id"]].append(ann)

    for img_id, anns in tqdm(ann_by_img.items(), desc=f"Converting {split}"):
        img_info = images[img_id]
        w, h = img_info["width"], img_info["height"]
        file_name = img_info["file_name"]
        file_stem = os.path.splitext(file_name)[0]
        label_path = os.path.join(yolo_root, f"labels/{split}/{file_stem}.txt")

        lines = []
        for ann in anns:
            # ✅ COCO: [x_min, y_min, width, height]
            x_min, y_min, box_w, box_h = ann["bbox"]

            x_center = (x_min + box_w / 2) / w
            y_center = (y_min + box_h / 2) / h
            box_w /= w
            box_h /= h

            # ✅ 잘못된 category_id 보정
            raw_id = ann["category_id"]
            cls_id = category_id_to_index.get(raw_id, 0)  # 존재하지 않으면 0으로 대체

            lines.append(f"{cls_id} {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}")

        with open(label_path, "w", encoding="utf-8") as lf:
            lf.write("\n".join(lines))

    print(f"✅ {split} 변환 완료 → {len(images)}개 이미지")

In [21]:
# ============================================
# ✅ COCO → YOLO 변환 파트 (기존 데이터 로딩 이후 추가)
# ============================================

import os, json
from tqdm import tqdm

# 기존 path와 train_dir 그대로 사용
train_json = os.path.join(path, "merged_coco_final_train.json")
val_json   = os.path.join(path, "merged_coco_final_valid.json")


save_coco_split_json(train_json, annotation_mapping_train, "train")
save_coco_split_json(val_json, annotation_mapping_val, "val")

# ✅ (2) YOLO 폴더 구조 생성
yolo_root = os.path.join(path, "yolo_dataset")
os.makedirs(os.path.join(yolo_root, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)


# ✅ (4) 변환 실행
coco_to_yolo(train_json, "train", yolo_root)
coco_to_yolo(val_json, "val", yolo_root)


# ✅ (5) data.yaml 생성
yaml_path = os.path.join(yolo_root, "data.yaml")
with open(train_json, "r", encoding="utf-8") as f:
    categories = json.load(f)["categories"]

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"train: {yolo_root}/images/train\n")
    f.write(f"val: {yolo_root}/images/val\n")
    f.write(f"nc: {len(categories)}\n")
    f.write("names:\n")
    for i, c in enumerate(categories):
        f.write(f"  {i}: {c['name']}\n")

print(f"🎯 data.yaml 생성 완료 → {yaml_path}")


✅ train JSON 저장 완료 → C:\Users\main\Downloads\final_dataset\merged_coco_final_train.json
✅ val JSON 저장 완료 → C:\Users\main\Downloads\final_dataset\merged_coco_final_valid.json


Converting train: 100%|██████████| 3052/3052 [00:00<00:00, 4848.97it/s]


✅ train 변환 완료 → 3052개 이미지


Converting val: 100%|██████████| 763/763 [00:00<00:00, 5085.48it/s]

✅ val 변환 완료 → 763개 이미지
🎯 data.yaml 생성 완료 → C:\Users\main\Downloads\final_dataset\yolo_dataset\data.yaml


In [26]:
import os, shutil

def copy_images(label_paths, split, img_src):
    copied = 0
    for label_path in label_paths:
        file_stem = os.path.splitext(os.path.basename(label_path))[0]
        for ext in [".jpg", ".png", ".jpeg"]:
            img_path = os.path.join(img_src, file_stem + ext)
            if os.path.exists(img_path):
                dst_path = os.path.join(yolo_root, f"images/{split}", os.path.basename(img_path))
                shutil.copy2(img_path, dst_path)
                copied += 1
                break
    print(f"✅ {split} 이미지 복사 완료: {copied}개")

In [27]:
import os, glob

img_src  = os.path.join(path, "images")         # 원본 이미지 폴더

# 대상 폴더 생성
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)

# train/val 라벨에 해당하는 파일 목록 읽기
train_labels = glob.glob(os.path.join(yolo_root, "labels/train", "*.txt"))
val_labels   = glob.glob(os.path.join(yolo_root, "labels/val", "*.txt"))

# 각 라벨에 대응하는 이미지 파일 복사
copy_images(train_labels, "train", img_src)
copy_images(val_labels, "val", img_src)

✅ train 이미지 복사 완료: 3052개
✅ val 이미지 복사 완료: 763개


In [1]:
import pandas as pd
import os
import torch

# torch.serialization.add_safe_globals([__import__("ultralytics").nn.tasks.DetectionModel])

# _orig_load = torch.load
# def _patched_load(*args, **kwargs):
#     kwargs.setdefault("weights_only", False)
#     return _orig_load(*args, **kwargs)
# torch.load = _patched_load

from ultralytics import YOLO

model_size = "s"
model_path = f"yolo11{model_size}.pt"
run_name = f"predict_{model_size}"
project_dir = "C:/content/runs/detect"
path = "C:/Users/main/Downloads/final_dataset"
yolo_root = os.path.join(path, "yolo_dataset")
data_path = os.path.join(yolo_root, 'data.yaml')

# ✅ 모델 로드 및 학습
model = YOLO(model_path)
results = model.train(
    data=data_path,
    epochs=200,
    imgsz=640,
    conf=0.25,
    iou=0.5,
    save=True,
    save_txt=True,
    save_conf=True,
    verbose=True,
    project=project_dir,
    name=run_name,   # ⭐ 사이즈에 따라 결과 폴더 분리됨
    exist_ok=True,
    workers=8,
    # preload=True,
    cache='ram',
    device=0,    
)

# ✅ 학습 완료 후 CSV → JSON 저장
# csv_path = os.path.join(project_dir, run_name, "results.csv")
# json_path = os.path.join(project_dir, run_name, "results.json")

# if os.path.exists(csv_path):
#     df = pd.read_csv(csv_path)
#     df_selected = df[["      box_loss", "   cls_loss", "   dfl_loss", "     mAP50", "  mAP50-95"]]
#     df_selected.columns = [col.strip() for col in df_selected.columns]  # 열 이름 정리
#     df_selected.to_json(json_path, orient="records", indent=2)
#     print(f"✅ JSON 저장 완료: {json_path}")
# else:
#     print("❌ results.csv 파일을 찾을 수 없습니다.")


New https://pypi.org/project/ultralytics/8.3.224 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.223  Python-3.11.4 torch-2.10.0.dev20251028+cu128 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=ram, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/main/Downloads/final_dataset\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.5, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_sc

c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py

                   all        763       3008      0.999      0.999      0.995      0.995
            100mg         53         53          1          1      0.995      0.995
                      49         49          1          1      0.995      0.995
        ()         32         32          1          1      0.995      0.995
    ()()         56         56          1          1      0.995      0.995
          500/20mg         30         30          1          1      0.995      0.995
              5mg         47         47          1          1      0.995      0.995
               10mg         34         34          1          1      0.995      0.994
          ()         46         46          1          1      0.995      0.995
          10mg/         51         51          1          1      0.995      0.995
          800mg         29         29          1          1      0.995      0.993
             20mg         27         27          1          1      0.995      0.995
         30mg

# 2차 학습

In [20]:
path_2 = "C:/Users/main/Downloads/dataset_v2"
train_dir_2 = os.path.join(path_2, "images")
annotation_dir_2 = os.path.join(path_2, "merged_annotations.json")

In [ ]:
img_files_2 = [f for f in os.listdir(train_dir_2) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
print(f"Train 수: {len(img_files_2)}")

Train 수: 3176


In [21]:
# JSON 로드
import json
from collections import defaultdict

with open(annotation_dir_2, "r", encoding="utf-8") as f:
    coco_data = json.load(f)

# ✅ 이미지별 어노테이션 매핑 생성
annotation_mapping_2 = defaultdict(list)
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    image_name = next(
        (img["file_name"] for img in coco_data["images"] if img["id"] == image_id),
        None
    )
    if image_name:
        annotation_mapping_2[image_name].append(ann)

print("✅ annotation_mapping 생성 완료")
print("예시 키:", list(annotation_mapping_2.keys())[:5])
print("첫 번째 이미지의 어노테이션 수:", len(annotation_mapping_2[list(annotation_mapping_2.keys())[0]]))


✅ annotation_mapping 생성 완료
예시 키: ['K-001900-016548-018110-021026_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-021026_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-027926_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_70_000_200.png', 'K-001900-016548-018110-029345_0_2_0_2_90_000_200.png']
첫 번째 이미지의 어노테이션 수: 4


In [22]:
# 전체 이미지 파일 이름 리스트
image_names_2 = list(annotation_mapping_2.keys())

# 어노테이션 매핑도 함께 나누기
annotation_mapping_train = {name: annotation_mapping_2[name] for name in image_names_2 }

print("✅ train/val 분리 완료")
print(f"Train 이미지 수: {len(annotation_mapping_train)}")

✅ train/val 분리 완료
Train 이미지 수: 644


In [24]:
# ============================================
# ✅ COCO → YOLO 변환 파트 (기존 데이터 로딩 이후 추가)
# ============================================
import os, json
from tqdm import tqdm

# 기존 path와 train_dir 그대로 사용
train_json = os.path.join(path_2, "merged_coco_final_train.json")
val_json   = os.path.join(path_2, "merged_coco_final_valid.json")

save_coco_split_json(train_json, annotation_mapping_train, "train")
save_coco_split_json(val_json, annotation_mapping_val, "val")

# ✅ (2) YOLO 폴더 구조 생성
yolo_root = os.path.join(path_2, "yolo_dataset")
os.makedirs(os.path.join(yolo_root, "labels/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "labels/val"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)

# ✅ (4) 변환 실행
coco_to_yolo(train_json, "train", yolo_root)
coco_to_yolo(val_json, "val", yolo_root)


# ✅ (5) data.yaml 생성
yaml_path = os.path.join(yolo_root, "data.yaml")
with open(train_json, "r", encoding="utf-8") as f:
    categories = json.load(f)["categories"]

with open(yaml_path, "w", encoding="utf-8") as f:
    f.write(f"train: {yolo_root}/images/train\n")
    f.write(f"val: {yolo_root}/images/val\n")
    f.write(f"nc: {len(categories)}\n")
    f.write("names:\n")
    for i, c in enumerate(categories):
        f.write(f"  {i}: {c['name']}\n")

print(f"🎯 data.yaml 생성 완료 → {yaml_path}")


✅ train JSON 저장 완료 → C:/Users/main/Downloads/dataset_v2\merged_coco_final_train.json
✅ val JSON 저장 완료 → C:/Users/main/Downloads/dataset_v2\merged_coco_final_valid.json


Converting train: 100%|██████████| 644/644 [00:00<00:00, 3565.38it/s]


✅ train 변환 완료 → 644개 이미지


Converting val: 100%|██████████| 146/146 [00:00<00:00, 3684.73it/s]

✅ val 변환 완료 → 146개 이미지
🎯 data.yaml 생성 완료 → C:/Users/main/Downloads/dataset_v2\yolo_dataset\data.yaml


In [27]:
import os, glob


img_src  = os.path.join(path_2, "images")         # 원본 이미지 폴더

# 대상 폴더 생성
os.makedirs(os.path.join(yolo_root, "images/train"), exist_ok=True)
os.makedirs(os.path.join(yolo_root, "images/val"), exist_ok=True)

# train/val 라벨에 해당하는 파일 목록 읽기
train_labels = glob.glob(os.path.join(yolo_root, "labels/train", "*.txt"))
val_labels   = glob.glob(os.path.join(yolo_root, "labels/val", "*.txt"))

# 각 라벨에 대응하는 이미지 파일 복사
copy_images(train_labels, "train", img_src)
copy_images(val_labels, "val", img_src)


✅ train 이미지 복사 완료: 644개
✅ val 이미지 복사 완료: 146개


# 학습

## YOLO11s

In [2]:
import time
import json
import pandas as pd
import os
import torch

# torch.serialization.add_safe_globals([__import__("ultralytics").nn.tasks.DetectionModel])

# _orig_load = torch.load
# def _patched_load(*args, **kwargs):
#     kwargs.setdefault("weights_only", False)
#     return _orig_load(*args, **kwargs)
# torch.load = _patched_load

from ultralytics import YOLO

path_2 = "C:/Users/main/Downloads/dataset_v2"
yolo_root = os.path.join(path_2, "yolo_dataset")
data_path = os.path.join(yolo_root, 'data.yaml')

# 기존 모델 로드 (best.pt)
base_model_dir = "C:/content/runs/detect/predict_s/weights"
model_path = os.path.join(base_model_dir, "best.pt")

# 새 프로젝트 설정
model_size = "s"
run_name = f"predict_{model_size}_fine_2"  # ✅ 덮어쓰기 방지
project_dir = "C:/content/runs/detect"

# 시간 측정
start_time = time.time()

# 모델 불러오기 및 재학습
model = YOLO(model_path)
results = model.train(
    data=data_path,
    epochs=100,  # 재학습은 보통 적은 에폭으로
    imgsz=640,
    # batch=8,
    project=project_dir,
    name=run_name,
    augment=False,
    exist_ok=True
)

# 학습 시간 기록
elapsed_minutes = round((time.time() - start_time) / 60, 2)

# 결과 정리
csv_path = os.path.join(project_dir, run_name, "results.csv")
json_path = os.path.join(project_dir, run_name, "results.json")

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    df.columns = [col.strip() for col in df.columns]
    keys = ["box_loss", "cls_loss", "dfl_loss", "mAP50", "mAP50-95"]
    df_selected = df[[k for k in keys if k in df.columns]]

    output = {
        "model": model_path,
        "epochs": len(df),
        "elapsed_minutes": elapsed_minutes,
        "logs": df_selected.to_dict(orient="records")
    }

    with open(json_path, "w") as f:
        json.dump(output, f, indent=2)

    print(f"✅ JSON 저장 완료: {json_path}")
else:
    print("❌ results.csv 파일을 찾을 수 없습니다.")


New https://pypi.org/project/ultralytics/8.3.224 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.223  Python-3.11.4 torch-2.10.0.dev20251028+cu128 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/main/Downloads/dataset_v2\yolo_dataset\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:/content/runs/detect/predict_s/weights\best.

c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 48148 (\N{HANGUL SYLLABLE BA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 53664 (\N{HANGUL SYLLABLE TO}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py:582: UserWarning: Glyph 54028 (\N{HANGUL SYLLABLE PA}) missing from font(s) DejaVu Sans.
  fig.savefig(plot_fname, dpi=250)
c:\Users\main\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\utils\metrics.py

                   all        146        540      0.998      0.999      0.995      0.995
            100mg         16         16          1          1      0.995      0.995
                       6          6          1          1      0.995      0.995
        ()          4          4          1          1      0.995      0.995
    ()()         56         56      0.998          1      0.995      0.995
          500/20mg          2          2          1          1      0.995      0.995
              5mg         14         14          1          1      0.995      0.995
               10mg          2          2          1          1      0.995      0.995
          ()          5          5          1          1      0.995      0.995
          10mg/          4          4          1          1      0.995      0.995
          800mg         10         10          1          1      0.995      0.995
             20mg          4          4          1          1      0.995      0.995
         30mg

# json 파일 저장

In [3]:
import os
import re
import json
import time
import math
import pandas as pd
from glob import glob
from PIL import Image
from ultralytics import YOLO

# =========================
# 0) 경로 설정 (필요에 맞게 수정)
# =========================
# 학습된 모델(best.pt)
best_model_path = r"C:\content\runs\detect\predict_s\weights\best.pt"

# 테스트 이미지 폴더
test_dir = r"C:\ai05-level1-project\test_images"

# 예측 산출물 폴더
save_dir = r"C:\Users\main\Downloads\pre"
run_name = "test_pred"
os.makedirs(save_dir, exist_ok=True)

# (중요) COCO train json: cls index → 실제 category_id 매핑용
#   - 당신이 YOLO 데이터셋 만든 출발점 COCO JSON(훈련용)을 지정하세요.
#   - categories 배열의 순서가 YOLO 클래스 인덱스와 동일해야 합니다.
train_coco_json = r"C:\Users\main\Downloads\dataset_v2\merged_coco_final_train.json"

# 최종 제출 CSV 경로
output_csv = os.path.join(save_dir, "200epo_1_0.0001.csv")

# =========================
# 1) 모델 로드 & 예측 수행
# =========================
model = YOLO(best_model_path)

start_time = time.time()
results = model.predict(
    source=test_dir,
    save_txt=True,     # labels/*.txt 저장
    save_conf=True,    # conf 포함
    project=save_dir,
    name=run_name,
    exist_ok=True,
    imgsz=640,
    conf=0.0001
)
elapsed = time.time() - start_time
print(f"✅ Prediction finished in {elapsed/60:.2f} min")

# =========================
# 2) cls index → COCO category_id 매핑표 만들기
# =========================
with open(train_coco_json, "r", encoding="utf-8") as f:
    coco = json.load(f)

# categories 배열의 순서가 YOLO 학습 시 사용된 names 순서와 같다고 가정
# 예: {0: categories[0]['id'], 1: categories[1]['id'], ...}
cls_to_catid = {i: cat["id"] for i, cat in enumerate(coco["categories"])}

# =========================
# 3) 이미지 크기 캐시 (픽셀 변환용)
# =========================
# YOLO txt는 (xc, yc, w, h) 정규화 값 → 픽셀 변환하려면 (W,H) 필요
def get_image_size(img_path):
    with Image.open(img_path) as im:
        return im.size  # (W, H)

# 이미지 파일명 → (W,H) 캐시
size_cache = {}

# =========================
# 4) YOLO txt → 타깃 CSV 변환
# =========================
labels_dir = os.path.join(save_dir, run_name, "labels")
txt_paths = sorted(glob(os.path.join(labels_dir, "*.txt")))

rows = []
annotation_id = 1

for txt_path in txt_paths:
    file_stem = os.path.basename(txt_path)[:-4]  # e.g., 'image_001'
    # image_id: 파일명에서 숫자 추출 (없으면 순번)
    m = re.findall(r"\d+", file_stem)
    image_id = int(m[0]) if m else annotation_id

    # 원본 이미지 경로 추정 (jpg/png 둘 다 탐색)
    candidates = [
        os.path.join(test_dir, file_stem + ext)
        for ext in [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp"]
    ]
    img_path = next((p for p in candidates if os.path.exists(p)), None)
    if img_path is None:
        # 이미지 못 찾으면 스킵
        print(f"⚠️ Image not found for {file_stem}, skipping.")
        continue

    if img_path not in size_cache:
        size_cache[img_path] = get_image_size(img_path)
    W, H = size_cache[img_path]

    with open(txt_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 6:
                continue
            cls = int(parts[0])
            xc, yc, w, h, conf = map(float, parts[1:6])

            # (xc,yc,w,h) [0~1] → 좌상단 (x,y), 가로세로 (w,h) [픽셀]
            px_w = w * W
            px_h = h * H
            px_x = (xc * W) - (px_w / 2.0)
            px_y = (yc * H) - (px_h / 2.0)

            # 음수/경계값 보정 + 정수화
            x0 = max(0, round(px_x))
            y0 = max(0, round(px_y))
            ww = max(0, round(px_w))
            hh = max(0, round(px_h))

            # cls index → 실제 category_id
            category_id = cls_to_catid.get(cls, cls)  # 없으면 원래 cls를 fallback

            rows.append({
                "annotation_id": annotation_id,
                "image_id": image_id,
                "category_id": category_id,
                "bbox_x": x0,
                "bbox_y": y0,
                "bbox_w": ww,
                "bbox_h": hh,
                "score": conf
            })
            annotation_id += 1

# =========================
# 5) CSV 저장 (컬럼/순서 고정)
# =========================
cols = ["annotation_id", "image_id", "category_id",
        "bbox_x", "bbox_y", "bbox_w", "bbox_h", "score"]
df = pd.DataFrame(rows, columns=cols)
df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"✅ Saved CSV: {output_csv}")
print(df.head())



image 1/843 C:\ai05-level1-project\test_images\1.png: 640x512 1  10mg, 2  800mgs, 1  5mg, 3 s, 5.6ms
image 2/843 C:\ai05-level1-project\test_images\10.png: 640x512 1  100mg, 1  20mg, 1 , 1  4mg, 1  5mg, 5.3ms
image 3/843 C:\ai05-level1-project\test_images\100.png: 640x512 1  100mg, 1  30mg, 1  5mg, 1 , 4.4ms
image 4/843 C:\ai05-level1-project\test_images\1003.png: 640x512 1 ()(), 1  20mg, 1  50/1000mg, 1  40/5mg, 5.3ms
image 5/843 C:\ai05-level1-project\test_images\1004.png: 640x512 1 ()(), 1  20mg, 1  50/1000mg, 1  40/5mg, 5.2ms
image 6/843 C:\ai05-level1-project\test_images\1005.png: 640x512 1 ()(), 1  20mg, 1  50/1000mg, 1  40/5mg, 4.3ms
image 7/843 C:\ai05-level1-project\test_images\1006.png: 640x512 1 ()(), 1  5/100mg, 1  50/850mg, 1  40/5mg, 1  75mg, 4.2ms
image 8/843 C:\ai05-level1-project\test_images\1007.png: 640x512 1 ()(), 1  5/100mg, 1  50/850mg, 1  40/5mg, 1  75mg, 4.1ms
image 9/843 C:\ai05-level1-project\test_images\1009.png: 640x512 1 ()(), 1  5/100mg, 1  50mg, 1  20mg,